In [1]:
import networkx as nx
import pickle
import math

In [2]:
targetDict={"NA/1":"1987-1989","NA/2":"2001-2003","NA/3":"2008-2010","NA/4":"2014-2016"}
carbA=[6,41,53]
carbB=[9,10,23,24,25,34,36,42,44,50,11]

In [3]:
def getFront(county):
    return '-'.join(county.split('-')[0:-1])

def NANONNA_Condition(node1,node2,filterTo):
    return (getFront(node1) in filterTo and getFront(node2) not in filterTo) or (getFront(node1) not in filterTo and getFront(node2) in filterTo)

def NANA_Condition(node1,node2,filterTo):
    return (getFront(node1) in filterTo and getFront(node2) in filterTo)


def getNASites(NAList,nodeNames):
    temp=[]
    for i in nodeNames:
        if 'CC' not in i:
            if '{}-{}'.format(int(i.split('-')[0]),int(i.split('-')[1])) in NAList:
                temp.append(i)
    return temp

def calCorrDistribution(A,G,nodeNames,filename,filterTo):
    '''N: Number of Nodes
    L: Number of Edges while considering weight
    k: Average degree'''

    yr=filename.split('_')[-1]
    L=0.0
    N=G.number_of_nodes()
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if NANA_Condition(node1,node2,filterTo):
                if node1 not in skip and node2 not in skip:
                    if A[i][j]!=0.0:
                        L=L+A[i][j]

    L=L/2
    print(L)
    print("Avg Clustering",nx.average_clustering(G,weight='weight'))
    try:
        k=2*L/N
        CC1=(2*L)/(N*(N-1))
        CC2=nx.average_clustering(G,weight='weight')
        randomDiameter=math.log(N)/math.log(k)
    except Exception as e:
        k=e
        CC1=e
        CC2=e
        randomDiameter=e

    try:
        diameter=nx.diameter(G)
        radius=nx.radius(G)
    except Exception as e:
        diameter=e
        radius=e

    temp=[yr,N,L,k,diameter,radius,randomDiameter,CC1,CC2]
    print(temp)

In [16]:

for target in targetDict:
    targetWrite=targetDict[target]
    endyear=targetWrite.split('-')[1]
    waveMatrix=pickle.load(open("{}/waveMatrix.pickle".format(target),"rb"))
    G=pickle.load(open("{}/Graph.pickle".format(target),"rb"))
    degree=pickle.load(open("{}/degree.pickle".format(target),"rb"))
    skip=pickle.load(open("{}/skip.pickle".format(target),"rb"))
    locations=pickle.load(open("locations.pickle","rb"))
    nodeNames=pickle.load(open("{}/Correlations/o3/{}/nodeNames.pickle".format(target,endyear),"rb"))
    NAData=pickle.load(open('{}/NAShrink.pickle'.format(target),'rb'))
    NAList=['{}-{}'.format(int(i.split('/')[0]),int(i.split('/')[1])) for i in NAData]
    NASites=getNASites(NAList,nodeNames)
    print(len(NASites))
    GNA=G.subgraph(NASites)
 
    print(target)
    stats(waveMatrix,G,nodeNames,NASites)
    print("CARB-A")
    NACarbA=[i for i in NASites if int(i.split('-')[0]) in carbA]
    statsfilter2(waveMatrix,G,nodeNames,NACarbA,carbA)
    
    print("CARB-B")
    NACarbB=[i for i in NASites if int(i.split('-')[0]) in carbB]
    statsfilter2(waveMatrix,G,nodeNames,NACarbB,carbB)
    
    print("Others")
    NAOthers=[i for i in NASites if int(i.split('-')[0]) not in carbB and int(i.split('-')[0]) not in carbA]
    statsfilter3(waveMatrix,G,nodeNames,NAOthers,carbA+carbB)
 
    

265
NA/1
Complete 0.311241872513
NA-NA 0.393246729385
Non NA-Non NA 0.294974238805
0.4003021148036254 0.5996978851963746 0.48012066337474096
Homophily= 0.449205959338
CARB-A
Complete 0.476276795
NA-NA 0.590263694471
Non NA-Non NA 0.400124571994
0.6458333333333334 0.3541666666666667 0.45746527777777785
Homophily= 0.381022796103
CARB-B
Complete 0.717675601404
NA-NA 0.732499993277
Non NA-Non NA 0.79808745432
0.8389830508474576 0.16101694915254236 0.2701809824762999
Homophily= 0.255198827031
Others
Complete 0.340840766975
NA-NA 0.388810094498
Non NA-Non NA 0.347574691745
0.1825 0.8175 0.29838749999999997
Homophily= 0.281304291068
279
NA/2
Complete 0.258046812103
NA-NA 0.409257840712
Non NA-Non NA 0.245893813296
0.25112511251125114 0.7488748874887489 0.37612258075492516
Homophily= 0.360262704615
CARB-A
Complete 0.469562749919
NA-NA 0.61479829644
Non NA-Non NA 0.366994096149
0.6307692307692307 0.36923076923076925 0.46579881656804734
Homophily= 0.375027427641
CARB-B
Complete 0.690091304163
NA

In [15]:
def stats(A,G,nodeNames,NAList):
    '''N: Number of Nodes
    L: Number of Edges while considering weight
    k: Average degree'''
    L=0.0
    N=G.number_of_nodes()
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if node1 not in skip and node2 not in skip:
                if A[i][j]!=0.0:
                    L=L+A[i][j]

    L=L/2
    print("Complete",(2*L)/(N*(N-1)))
    
    L=0.0
    N=len(NAList)
    st=0
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if node1 not in skip and node2 not in skip:
                if node1 in NAList and node2 in NAList:
                    if A[i][j]!=0.0:
                        L=L+A[i][j]

    L=L/2
   
    print("NA-NA",(2*L)/(N*(N-1)))
    
    L=0.0
    N=0
    for node in nodeNames:
        if node not in NAList:
            N+=1
    
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if node1 not in skip and node2 not in skip:
                if node1 not in NAList and node2 not in NAList:
                    if A[i][j]!=0.0:
                        L=L+A[i][j]

    L=L/2
    
    print("Non NA-Non NA",(2*L)/(N*(N-1)))
    
    # Homophily
    p=len(NAList)/len(nodeNames)
    q=len(list(set(nodeNames)-set(NAList)))/len(nodeNames)
    print(p,q,2*p*q)
    
    L=0.0
    L_Total=0.0
    N=G.number_of_nodes()
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if node1 not in skip and node2 not in skip:
                L_Total+=A[i][j]
                if (node1 in NAList and node2 not in NAList) or (node1 not in NAList and node2 in NAList):
                    if A[i][j]!=0.0:
                        L=L+A[i][j]
                        
    print("Homophily=",L/L_Total)                  
    

def statsfilter2(A,G,nodeNames,NAList,filter2):
    '''N: Number of Nodes
    L: Number of Edges while considering weight
    k: Average degree'''
    L=0.0
    N=len([i for i in nodeNames if 'CC' not in i if int(i.split('-')[0]) in filter2])
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if node1 not in skip and node2 not in skip:
                if int(node1.split('-')[0]) in filter2 and int(node2.split('-')[0]) in filter2:
                    if A[i][j]!=0.0:
                        L=L+A[i][j]

    L=L/2
    if N!=0:
        print("Complete",(2*L)/(N*(N-1)))
    
    L=0.0
    N=len(NAList)
    st=0
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if node1 not in skip and node2 not in skip:
                    if node1 in NAList and node2 in NAList:
                        if A[i][j]!=0.0:
                            L=L+A[i][j]

    L=L/2
    if N!=0:
        print("NA-NA",(2*L)/(N*(N-1)))
    
    L=0.0
    N=0
    for node in nodeNames:
        if 'CC' not in node:
            if int(node.split('-')[0]) in filter2:
                if node not in NAList:
                    N+=1

    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if node1 not in skip and node2 not in skip:
                if int(node1.split('-')[0]) in filter2 and int(node2.split('-')[0]) in filter2:
                    if node1 not in NAList and node2 not in NAList:
                        if A[i][j]!=0.0:
                            L=L+A[i][j]

    L=L/2
    
    if N!=0:
        print("Non NA-Non NA",(2*L)/(N*(N-1)))
    
    # Homophily
    N=len([i for i in nodeNames if 'CC' not in i if int(i.split('-')[0]) in filter2])
    p=len(NAList)/N
    nodeNamesA=[i for i in nodeNames if 'CC' not in i if int(i.split('-')[0]) in filter2]
    q=len(list(set(nodeNamesA)-set(NAList)))/N
    print(p,q,2*p*q)
    
    L=0.0
    L_Total=0.0
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if node1 not in skip and node2 not in skip:
                if int(node1.split('-')[0]) in filter2 and int(node2.split('-')[0]) in filter2:
                    L_Total+=A[i][j]
                    if (node1 in NAList and node2 not in NAList) or (node1 not in NAList and node2 in NAList):
                        if A[i][j]!=0.0:
                            L=L+A[i][j]

    print("Homophily=",L/L_Total)               

    
def statsfilter3(A,G,nodeNames,NAList,filter3):
    '''N: Number of Nodes
    L: Number of Edges while considering weight
    k: Average degree'''
    L=0.0
    N=len([i for i in nodeNames if 'CC' not in i if int(i.split('-')[0]) not in filter3])
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if node1 not in skip and node2 not in skip:
                if int(node1.split('-')[0]) not in filter3 and int(node2.split('-')[0]) not in filter3:
                    if A[i][j]!=0.0:
                        L=L+A[i][j]

    L=L/2
    if N!=0:
        print("Complete",(2*L)/(N*(N-1)))
    
    L=0.0
    N=len(NAList)
    st=0
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if node1 not in skip and node2 not in skip:
                if node1 in NAList and node2 in NAList:
                    if A[i][j]!=0.0:
                        L=L+A[i][j]

    L=L/2
   
    if N!=0:
        print("NA-NA",(2*L)/(N*(N-1)))
    
    L=0.0
    N=0
    
    for node in nodeNames:
        if 'CC' not in node:
            if int(node.split('-')[0]) not in filter3:
                if node not in NAList:
                    N+=1
    
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if node1 not in skip and node2 not in skip:
                if int(node1.split('-')[0]) not in filter3 and int(node2.split('-')[0]) not in filter3:
                    if node1 not in NAList and node2 not in NAList:
                        if A[i][j]!=0.0:
                            L=L+A[i][j]

    L=L/2
    
    if N!=0:
        print("Non NA-Non NA",(2*L)/(N*(N-1)))
    
    # Homophily
    N=len([i for i in nodeNames if 'CC' not in i if int(i.split('-')[0]) not in filter3])
    nodenamesOthers=[i for i in nodeNames if 'CC' not in i if int(i.split('-')[0]) not in filter3]
    p=len(NAList)/N
    q=len(list(set(nodenamesOthers)-set(NAList)))/N
    print(p,q,2*p*q)
    
    L=0.0
    L_Total=0.0
    
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            node1 = nodeNames[i]
            node2 = nodeNames[j]
            if node1 not in skip and node2 not in skip:
                if int(node1.split('-')[0]) not in filter3 and int(node2.split('-')[0]) not in filter3:
                    L_Total+=A[i][j]
                    if (node1 in NAList and node2 not in NAList) or (node1 not in NAList and node2 in NAList):
                        if A[i][j]!=0.0:
                            L=L+A[i][j]
                        
    print("Homophily=",L/L_Total)  
    
# def statsfilter2(A,G,nodeNames,NAList,filter2):
#     '''N: Number of Nodes
#     L: Number of Edges while considering weight
#     k: Average degree'''
#     L=0.0
#     N=len([i for i in nodeNames if 'CC' not in i if int(i.split('-')[0]) in filter2])
#     for i in range(0,len(A)):
#         for j in range(0,len(A)):
#             node1 = nodeNames[i]
#             node2 = nodeNames[j]
#             if node1 not in skip and node2 not in skip:
#                 if int(node1.split('-')[0]) in filter2 and int(node2.split('-')[0]) in filter2:
#                     if A[i][j]!=0.0:
#                         L=L+A[i][j]

#     L=L/2
#     print("Complete",(2*L)/(N*(N-1)))
    
#     L=0.0
#     N=len(NAList)
#     st=0
#     for i in range(0,len(A)):
#         for j in range(0,len(A)):
#             node1 = nodeNames[i]
#             node2 = nodeNames[j]
#             if node1 not in skip and node2 not in skip:
#                 if node1 in NAList and node2 in NAList:
#                     if A[i][j]!=0.0:
#                         L=L+A[i][j]

#     L=L/2
   
#     print("NA-NA",(2*L)/(N*(N-1)))
    
#     L=0.0
#     N=0
#     for node in nodeNames:
#         if node not in NAList:
#             N+=1
    
#     for i in range(0,len(A)):
#         for j in range(0,len(A)):
#             node1 = nodeNames[i]
#             node2 = nodeNames[j]
#             if node1 not in skip and node2 not in skip:
#                 if node1 not in NAList and node2 not in NAList:
#                     if A[i][j]!=0.0:
#                         L=L+A[i][j]

#     L=L/2
    
#     print("Non NA-Non NA",(2*L)/(N*(N-1)))
    
#     # Homophily
#     p=len(NAList)/len(nodeNames)
#     q=len(list(set(nodeNames)-set(NAList)))/len(nodeNames)
#     print(p,q,2*p*q)
    
#     L=0.0
#     L_Total=0.0
#     N=G.number_of_nodes()
#     for i in range(0,len(A)):
#         for j in range(0,len(A)):
#             node1 = nodeNames[i]
#             node2 = nodeNames[j]
#             if node1 not in skip and node2 not in skip:
#                 L_Total+=A[i][j]
#                 if (node1 in NAList and node2 not in NAList) or (node1 not in NAList and node2 in NAList):
#                     if A[i][j]!=0.0:
#                         L=L+A[i][j]
                        
#     print("Homophily=",L/L_Total)               

    
# def statsfilter3(A,G,nodeNames,NAList,filter3):
#     '''N: Number of Nodes
#     L: Number of Edges while considering weight
#     k: Average degree'''
#     L=0.0
#     N=len([i for i in nodeNames if 'CC' not in i if int(i.split('-')[0]) not in filter3])
#     for i in range(0,len(A)):
#         for j in range(0,len(A)):
#             node1 = nodeNames[i]
#             node2 = nodeNames[j]
#             if node1 not in skip and node2 not in skip:
#                 if int(node1.split('-')[0]) not in filter3 and int(node2.split('-')[0]) not in filter3:
#                     if A[i][j]!=0.0:
#                         L=L+A[i][j]

#     L=L/2
#     print("Complete",(2*L)/(N*(N-1)))
    
#     L=0.0
#     N=len(NAList)
#     st=0
#     for i in range(0,len(A)):
#         for j in range(0,len(A)):
#             node1 = nodeNames[i]
#             node2 = nodeNames[j]
#             if node1 not in skip and node2 not in skip:
#                 if node1 in NAList and node2 in NAList:
#                     if A[i][j]!=0.0:
#                         L=L+A[i][j]

#     L=L/2
   
#     print("NA-NA",(2*L)/(N*(N-1)))
    
#     L=0.0
#     N=0
#     for node in nodeNames:
#         if node not in NAList:
#             N+=1
    
#     for i in range(0,len(A)):
#         for j in range(0,len(A)):
#             node1 = nodeNames[i]
#             node2 = nodeNames[j]
#             if node1 not in skip and node2 not in skip:
#                 if node1 not in NAList and node2 not in NAList:
#                     if A[i][j]!=0.0:
#                         L=L+A[i][j]

#     L=L/2
    
#     print("Non NA-Non NA",(2*L)/(N*(N-1)))
    
#     # Homophily
#     p=len(NAList)/len(nodeNames)
#     q=len(list(set(nodeNames)-set(NAList)))/len(nodeNames)
#     print(p,q,2*p*q)
    
#     L=0.0
#     L_Total=0.0
#     N=G.number_of_nodes()
#     for i in range(0,len(A)):
#         for j in range(0,len(A)):
#             node1 = nodeNames[i]
#             node2 = nodeNames[j]
#             if node1 not in skip and node2 not in skip:
#                 L_Total+=A[i][j]
#                 if (node1 in NAList and node2 not in NAList) or (node1 not in NAList and node2 in NAList):
#                     if A[i][j]!=0.0:
#                         L=L+A[i][j]
                        
#     print("Homophily=",L/L_Total)  


In [18]:
G.number_of_edges?

In [12]:
nodeNames[1000:1111]

['6-67-11',
 '6-67-12',
 '6-67-13',
 '6-67-2',
 '6-67-5003',
 '6-67-6',
 '6-69-2',
 '6-69-3',
 '6-7-2',
 '6-7-7',
 '6-71-1',
 '6-71-1004',
 '6-71-12',
 '6-71-1234',
 '6-71-17',
 '6-71-2002',
 '6-71-306',
 '6-71-4001',
 '6-71-4003',
 '6-71-5',
 '6-71-9002',
 '6-71-9004',
 '6-73-1',
 '6-73-1001',
 '6-73-1002',
 '6-73-1006',
 '6-73-1007',
 '6-73-1008',
 '6-73-2007',
 '6-73-3',
 '6-73-6',
 '6-75-5',
 '6-77-1002',
 '6-77-3003',
 '6-79-2001',
 '6-79-2002',
 '6-79-3001',
 '6-79-4002',
 '6-79-5',
 '6-79-8001',
 '6-81-1001',
 '6-83-1008',
 '6-83-1013',
 '6-83-1014',
 '6-83-1018',
 '6-83-1021',
 '6-83-1025',
 '6-83-2004',
 '6-83-2011',
 '6-83-2012',
 '6-83-3001',
 '6-83-4003',
 '6-83-8',
 '6-85-1001',
 '6-85-2',
 '6-85-2005',
 '6-85-2006',
 '6-85-2007',
 '6-87-3',
 '6-87-4',
 '6-87-6',
 '6-87-7',
 '6-89-3003',
 '6-89-4',
 '6-9-1',
 '6-93-2001',
 '6-95-3002',
 '6-95-4',
 '6-97-1003',
 '6-97-3',
 '6-99-5',
 '6-99-6',
 '78-20-1',
 '8-1-3001',
 '8-13-11',
 '8-31-14',
 '8-35-2',
 '8-41-13',
 '8-5-2',

In [10]:
len(nodeNames)

1111